In [ ]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 20)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))

In [ ]:
import pandas as pd
X_train = pd.read_csv("data/train")

In [ ]:
X_test = pd.read_csv("data/test")

In [ ]:
X_test.head().site_id

In [ ]:
X_train['site_id_md5']=X_train.site_id.apply(string2numeric_hash)
X_train['site_domain_md5']=X_train.site_domain.apply(string2numeric_hash)
X_train['site_category_md5']=X_train.site_category.apply(string2numeric_hash)

X_train['app_id_md5']=X_train.app_id.apply(string2numeric_hash)
X_train['app_domain_md5']=X_train.app_domain.apply(string2numeric_hash)
X_train['app_category_md5']=X_train.app_category.apply(string2numeric_hash)

X_train['device_id_md5']=X_train.device_id.apply(string2numeric_hash)
X_train['device_ip_md5']=X_train.device_ip.apply(string2numeric_hash)
X_train['device_model_md5']=X_train.device_model.apply(string2numeric_hash)

In [ ]:
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)

In [ ]:
tmp_features = X_train[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
labels = X_train.click.values

In [ ]:
import xgboost as xgb
import numpy as np
param = {'max_depth':8, 'eta':0.1, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 4
param['colsample_bytree'] = 0.5
param['eval_metric'] = 'logloss'

#msk = np.random.rand(len(X_train)) < 0.8
msk = np.load("20170727_msk.npy")

dtrain = xgb.DMatrix(tmp_features[msk], labels[msk])
dvali = xgb.DMatrix(tmp_features[~msk], labels[~msk])

In [ ]:
num_round = 80000
evallist  = [ (dtrain,'train'),(dvali,'eval')]
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=500)

In [ ]:
bst.best_score

In [ ]:
bst.save_model('md_hash_0803.model')

# testing phase

In [ ]:
import pandas as pd
X_test = pd.read_csv("data/test")

In [ ]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))

In [ ]:
X_test['site_id_md5'] = X_test.site_id.apply(string2numeric_hash)
X_test['site_domain_md5'] = X_test.site_domain.apply(string2numeric_hash)
X_test['site_category_md5'] = X_test.site_category.apply(string2numeric_hash)

X_test['app_id_md5'] = X_test.app_id.apply(string2numeric_hash)
X_test['app_domain_md5'] = X_test.app_domain.apply(string2numeric_hash)
X_test['app_category_md5'] = X_test.app_category.apply(string2numeric_hash)

X_test['device_id_md5'] = X_test.device_id.apply(string2numeric_hash)
X_test['device_ip_md5'] = X_test.device_ip.apply(string2numeric_hash)
X_test['device_model_md5'] = X_test.device_model.apply(string2numeric_hash)

In [ ]:
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [ ]:
tmp_features = X_test[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
import xgboost as xgb

In [ ]:
bst = xgb.Booster({'nthread':4}) #init model
bst.load_model("md_hash_0803.model") # load data

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=500)

In [ ]:
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=250)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_250.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=1000)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_1000.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=1500)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_1500.csv",index=False)

# [Sec2] handle not interception training and testing as missing

In [ ]:
import pandas as pd
X_train = pd.read_csv("data/train")

In [ ]:
X_test = pd.read_csv("data/test")

## site_id

In [ ]:
site_id_train = X_train.site_id.value_counts()/len(X_train)
site_id_test = X_test.site_id.value_counts()/len(X_test)
intersection = site_id_test.index.intersection(site_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_id = X_train.site_id.apply(missing_replace)
X_test.site_id = X_test.site_id.apply(missing_replace)

## site_domain

In [ ]:
site_domain_train = X_train.site_domain.value_counts()/len(X_train)
site_domain_test = X_test.site_domain.value_counts()/len(X_test)
intersection = site_domain_test.index.intersection(site_domain_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_domain = X_train.site_domain.apply(missing_replace)
X_test.site_domain = X_test.site_domain.apply(missing_replace)

## site_category

In [ ]:
site_category_train = X_train.site_category.value_counts()/len(X_train)
site_category_test = X_test.site_category.value_counts()/len(X_test)
intersection = site_category_test.index.intersection(site_category_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_category = X_train.site_category.apply(missing_replace)
X_test.site_category = X_test.site_category.apply(missing_replace)

## app_id

In [ ]:
app_id_train = X_train.app_id.value_counts()/len(X_train)
app_id_test = X_test.app_id.value_counts()/len(X_test)
intersection = app_id_test.index.intersection(app_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_id = X_train.app_id.apply(missing_replace)
X_test.app_id = X_test.app_id.apply(missing_replace)

## app_domain

In [ ]:
app_domain_train = X_train.app_domain.value_counts()/len(X_train)
app_domain_test = X_test.app_domain.value_counts()/len(X_test)
intersection = app_domain_test.index.intersection(app_domain_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_domain = X_train.app_domain.apply(missing_replace)
X_test.app_domain = X_test.app_domain.apply(missing_replace)

## app_category

In [ ]:
app_category_train = X_train.app_category.value_counts()/len(X_train)
app_category_test = X_test.app_category.value_counts()/len(X_test)
intersection = app_category_test.index.intersection(app_category_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_category = X_train.app_category.apply(missing_replace)
X_test.app_category = X_test.app_category.apply(missing_replace)

## device_id

In [ ]:
device_id_train = X_train.device_id.value_counts()/len(X_train)
device_id_test = X_test.device_id.value_counts()/len(X_test)
intersection = device_id_test.index.intersection(device_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_id = X_train.device_id.apply(missing_replace)
X_test.device_id = X_test.device_id.apply(missing_replace)

## device_ip

In [ ]:
device_ip_train = X_train.device_ip.value_counts()/len(X_train)
device_ip_test = X_test.device_ip.value_counts()/len(X_test)
intersection = device_ip_test.index.intersection(device_ip_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_ip = X_train.device_ip.apply(missing_replace)
X_test.device_ip = X_test.device_ip.apply(missing_replace)

## device_model

In [ ]:
device_model_train = X_train.device_model.value_counts()/len(X_train)
device_model_test = X_test.device_model.value_counts()/len(X_test)
intersection = device_model_test.index.intersection(device_model_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_model = X_train.device_model.apply(missing_replace)
X_test.device_model = X_test.device_model.apply(missing_replace)

In [ ]:
X_train.to_pickle('data/train_xgb_miss')
X_test.to_pickle('data/test_xgb_miss')

## hash! 

In [1]:
import pandas as pd 
X_train = pd.read_pickle('data/train_xgb_miss')
X_test = pd.read_pickle('data/test_xgb_miss')

In [4]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    if text=='missing':
        return 0 # xgboost treat 0 as missing
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))
print (string2numeric_hash("missing"))

1962341389
1823841227
0


In [5]:
X_train['site_id_md5']=X_train.site_id.apply(string2numeric_hash)
X_train['site_domain_md5']=X_train.site_domain.apply(string2numeric_hash)
X_train['site_category_md5']=X_train.site_category.apply(string2numeric_hash)

X_train['app_id_md5']=X_train.app_id.apply(string2numeric_hash)
X_train['app_domain_md5']=X_train.app_domain.apply(string2numeric_hash)
X_train['app_category_md5']=X_train.app_category.apply(string2numeric_hash)

X_train['device_id_md5']=X_train.device_id.apply(string2numeric_hash)
X_train['device_ip_md5']=X_train.device_ip.apply(string2numeric_hash)
X_train['device_model_md5']=X_train.device_model.apply(string2numeric_hash)

In [6]:
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)

In [7]:
X_train_features = X_train[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [8]:
labels = X_train.click.values

In [9]:
import numpy as np
np.save('train_xgb_hash',X_train_features)
np.save('train_xgb_y',labels)

## Hash on Testing

In [14]:
X_test['site_id_md5'] = X_test.site_id.apply(string2numeric_hash)
X_test['site_domain_md5'] = X_test.site_domain.apply(string2numeric_hash)
X_test['site_category_md5'] = X_test.site_category.apply(string2numeric_hash)

X_test['app_id_md5'] = X_test.app_id.apply(string2numeric_hash)
X_test['app_domain_md5'] = X_test.app_domain.apply(string2numeric_hash)
X_test['app_category_md5'] = X_test.app_category.apply(string2numeric_hash)

X_test['device_id_md5'] = X_test.device_id.apply(string2numeric_hash)
X_test['device_ip_md5'] = X_test.device_ip.apply(string2numeric_hash)
X_test['device_model_md5'] = X_test.device_model.apply(string2numeric_hash)

In [15]:
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [16]:
X_test_features = X_test[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [17]:
import numpy as np
np.save('test_xgb_hash',X_test_features)

# [Sec3] Strart training XGB/lightgbm


In [1]:
import numpy as np
X_train_features = np.load('train_xgb_hash.npy')
labels = np.load('train_xgb_y.npy')

## [Sec3-1] LightGBM

In [19]:
import lightgbm as lgb
#create lightgbm dataset
msk = np.load("20170727_msk.npy")
lgb_train = lgb.Dataset(X_train_features[msk], labels[msk])
lgb_eval = lgb.Dataset(X_train_features[~msk], labels[~msk],reference=lgb_train)


In [20]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves': 31, # defauly leaves(31) amount for each tree
    'learning_rate': 0.01,
    'feature_fraction': 0.5, # will select 50% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=30000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

[21666]	valid_0's binary_logloss: 0.391061
[21667]	valid_0's binary_logloss: 0.391061
[21668]	valid_0's binary_logloss: 0.391061
[21669]	valid_0's binary_logloss: 0.391061
[21670]	valid_0's binary_logloss: 0.391061
[21671]	valid_0's binary_logloss: 0.391061
[21672]	valid_0's binary_logloss: 0.391061
[21673]	valid_0's binary_logloss: 0.391061
[21674]	valid_0's binary_logloss: 0.391061
[21675]	valid_0's binary_logloss: 0.391061
[21676]	valid_0's binary_logloss: 0.391061
[21677]	valid_0's binary_logloss: 0.391061
[21678]	valid_0's binary_logloss: 0.391061
[21679]	valid_0's binary_logloss: 0.391061
[21680]	valid_0's binary_logloss: 0.391061
[21681]	valid_0's binary_logloss: 0.39106
[21682]	valid_0's binary_logloss: 0.39106
[21683]	valid_0's binary_logloss: 0.39106
[21684]	valid_0's binary_logloss: 0.39106
[21685]	valid_0's binary_logloss: 0.39106
[21686]	valid_0's binary_logloss: 0.39106
[21687]	valid_0's binary_logloss: 0.39106
[21688]	valid_0's binary_logloss: 0.39106
[21689]	valid_0's b

[21858]	valid_0's binary_logloss: 0.391042
[21859]	valid_0's binary_logloss: 0.391042
[21860]	valid_0's binary_logloss: 0.391042
[21861]	valid_0's binary_logloss: 0.391042
[21862]	valid_0's binary_logloss: 0.391041
[21863]	valid_0's binary_logloss: 0.391041
[21864]	valid_0's binary_logloss: 0.391041
[21865]	valid_0's binary_logloss: 0.391041
[21866]	valid_0's binary_logloss: 0.391041
[21867]	valid_0's binary_logloss: 0.391041
[21868]	valid_0's binary_logloss: 0.391041
[21869]	valid_0's binary_logloss: 0.391041
[21870]	valid_0's binary_logloss: 0.391041
[21871]	valid_0's binary_logloss: 0.391041
[21872]	valid_0's binary_logloss: 0.391041
[21873]	valid_0's binary_logloss: 0.391041
[21874]	valid_0's binary_logloss: 0.39104
[21875]	valid_0's binary_logloss: 0.39104
[21876]	valid_0's binary_logloss: 0.39104
[21877]	valid_0's binary_logloss: 0.39104
[21878]	valid_0's binary_logloss: 0.39104
[21879]	valid_0's binary_logloss: 0.39104
[21880]	valid_0's binary_logloss: 0.39104
[21881]	valid_0's 

[22050]	valid_0's binary_logloss: 0.39102
[22051]	valid_0's binary_logloss: 0.39102
[22052]	valid_0's binary_logloss: 0.39102
[22053]	valid_0's binary_logloss: 0.39102
[22054]	valid_0's binary_logloss: 0.391019
[22055]	valid_0's binary_logloss: 0.391019
[22056]	valid_0's binary_logloss: 0.391019
[22057]	valid_0's binary_logloss: 0.391019
[22058]	valid_0's binary_logloss: 0.391019
[22059]	valid_0's binary_logloss: 0.391019
[22060]	valid_0's binary_logloss: 0.391019
[22061]	valid_0's binary_logloss: 0.391018
[22062]	valid_0's binary_logloss: 0.391018
[22063]	valid_0's binary_logloss: 0.391018
[22064]	valid_0's binary_logloss: 0.391018
[22065]	valid_0's binary_logloss: 0.391018
[22066]	valid_0's binary_logloss: 0.391018
[22067]	valid_0's binary_logloss: 0.391018
[22068]	valid_0's binary_logloss: 0.391018
[22069]	valid_0's binary_logloss: 0.391018
[22070]	valid_0's binary_logloss: 0.391018
[22071]	valid_0's binary_logloss: 0.391018
[22072]	valid_0's binary_logloss: 0.391017
[22073]	valid_0

[22242]	valid_0's binary_logloss: 0.390995
[22243]	valid_0's binary_logloss: 0.390995
[22244]	valid_0's binary_logloss: 0.390995
[22245]	valid_0's binary_logloss: 0.390995
[22246]	valid_0's binary_logloss: 0.390995
[22247]	valid_0's binary_logloss: 0.390995
[22248]	valid_0's binary_logloss: 0.390995
[22249]	valid_0's binary_logloss: 0.390995
[22250]	valid_0's binary_logloss: 0.390994
[22251]	valid_0's binary_logloss: 0.390994
[22252]	valid_0's binary_logloss: 0.390994
[22253]	valid_0's binary_logloss: 0.390994
[22254]	valid_0's binary_logloss: 0.390994
[22255]	valid_0's binary_logloss: 0.390994
[22256]	valid_0's binary_logloss: 0.390994
[22257]	valid_0's binary_logloss: 0.390994
[22258]	valid_0's binary_logloss: 0.390994
[22259]	valid_0's binary_logloss: 0.390993
[22260]	valid_0's binary_logloss: 0.390993
[22261]	valid_0's binary_logloss: 0.390993
[22262]	valid_0's binary_logloss: 0.390993
[22263]	valid_0's binary_logloss: 0.390993
[22264]	valid_0's binary_logloss: 0.390993
[22265]	val

[22433]	valid_0's binary_logloss: 0.390972
[22434]	valid_0's binary_logloss: 0.390972
[22435]	valid_0's binary_logloss: 0.390972
[22436]	valid_0's binary_logloss: 0.390972
[22437]	valid_0's binary_logloss: 0.390972
[22438]	valid_0's binary_logloss: 0.390972
[22439]	valid_0's binary_logloss: 0.390972
[22440]	valid_0's binary_logloss: 0.390972
[22441]	valid_0's binary_logloss: 0.390972
[22442]	valid_0's binary_logloss: 0.390972
[22443]	valid_0's binary_logloss: 0.390972
[22444]	valid_0's binary_logloss: 0.390972
[22445]	valid_0's binary_logloss: 0.390972
[22446]	valid_0's binary_logloss: 0.390971
[22447]	valid_0's binary_logloss: 0.390971
[22448]	valid_0's binary_logloss: 0.390971
[22449]	valid_0's binary_logloss: 0.390971
[22450]	valid_0's binary_logloss: 0.390971
[22451]	valid_0's binary_logloss: 0.390971
[22452]	valid_0's binary_logloss: 0.390971
[22453]	valid_0's binary_logloss: 0.390971
[22454]	valid_0's binary_logloss: 0.390971
[22455]	valid_0's binary_logloss: 0.390971
[22456]	val

...

[29133]	valid_0's binary_logloss: 0.390319
[29134]	valid_0's binary_logloss: 0.390319
[29135]	valid_0's binary_logloss: 0.390319
[29136]	valid_0's binary_logloss: 0.390319
[29137]	valid_0's binary_logloss: 0.390319
[29138]	valid_0's binary_logloss: 0.390319
[29139]	valid_0's binary_logloss: 0.390319
[29140]	valid_0's binary_logloss: 0.390319
[29141]	valid_0's binary_logloss: 0.390319
[29142]	valid_0's binary_logloss: 0.390319
[29143]	valid_0's binary_logloss: 0.390319
[29144]	valid_0's binary_logloss: 0.390319
[29145]	valid_0's binary_logloss: 0.390319
[29146]	valid_0's binary_logloss: 0.390319
[29147]	valid_0's binary_logloss: 0.390319
[29148]	valid_0's binary_logloss: 0.390318
[29149]	valid_0's binary_logloss: 0.390318
[29150]	valid_0's binary_logloss: 0.390318
[29151]	valid_0's binary_logloss: 0.390318
[29152]	valid_0's binary_logloss: 0.390318
[29153]	valid_0's binary_logloss: 0.390318
[29154]	valid_0's binary_logloss: 0.390318
[29155]	valid_0's binary_logloss: 0.390318
[29156]	val

[29324]	valid_0's binary_logloss: 0.390306
[29325]	valid_0's binary_logloss: 0.390306
[29326]	valid_0's binary_logloss: 0.390306
[29327]	valid_0's binary_logloss: 0.390306
[29328]	valid_0's binary_logloss: 0.390305
[29329]	valid_0's binary_logloss: 0.390305
[29330]	valid_0's binary_logloss: 0.390305
[29331]	valid_0's binary_logloss: 0.390305
[29332]	valid_0's binary_logloss: 0.390305
[29333]	valid_0's binary_logloss: 0.390305
[29334]	valid_0's binary_logloss: 0.390305
[29335]	valid_0's binary_logloss: 0.390305
[29336]	valid_0's binary_logloss: 0.390305
[29337]	valid_0's binary_logloss: 0.390305
[29338]	valid_0's binary_logloss: 0.390305
[29339]	valid_0's binary_logloss: 0.390305
[29340]	valid_0's binary_logloss: 0.390304
[29341]	valid_0's binary_logloss: 0.390304
[29342]	valid_0's binary_logloss: 0.390304
[29343]	valid_0's binary_logloss: 0.390304
[29344]	valid_0's binary_logloss: 0.390304
[29345]	valid_0's binary_logloss: 0.390304
[29346]	valid_0's binary_logloss: 0.390304
[29347]	val

[29516]	valid_0's binary_logloss: 0.390293
[29517]	valid_0's binary_logloss: 0.390293
[29518]	valid_0's binary_logloss: 0.390293
[29519]	valid_0's binary_logloss: 0.390293
[29520]	valid_0's binary_logloss: 0.390293
[29521]	valid_0's binary_logloss: 0.390293
[29522]	valid_0's binary_logloss: 0.390293
[29523]	valid_0's binary_logloss: 0.390293
[29524]	valid_0's binary_logloss: 0.390292
[29525]	valid_0's binary_logloss: 0.390292
[29526]	valid_0's binary_logloss: 0.390292
[29527]	valid_0's binary_logloss: 0.390292
[29528]	valid_0's binary_logloss: 0.390292
[29529]	valid_0's binary_logloss: 0.390292
[29530]	valid_0's binary_logloss: 0.390292
[29531]	valid_0's binary_logloss: 0.390292
[29532]	valid_0's binary_logloss: 0.390291
[29533]	valid_0's binary_logloss: 0.390291
[29534]	valid_0's binary_logloss: 0.390291
[29535]	valid_0's binary_logloss: 0.390291
[29536]	valid_0's binary_logloss: 0.390291
[29537]	valid_0's binary_logloss: 0.390291
[29538]	valid_0's binary_logloss: 0.390291
[29539]	val

[29707]	valid_0's binary_logloss: 0.390276
[29708]	valid_0's binary_logloss: 0.390276
[29709]	valid_0's binary_logloss: 0.390276
[29710]	valid_0's binary_logloss: 0.390276
[29711]	valid_0's binary_logloss: 0.390276
[29712]	valid_0's binary_logloss: 0.390276
[29713]	valid_0's binary_logloss: 0.390276
[29714]	valid_0's binary_logloss: 0.390276
[29715]	valid_0's binary_logloss: 0.390276
[29716]	valid_0's binary_logloss: 0.390276
[29717]	valid_0's binary_logloss: 0.390276
[29718]	valid_0's binary_logloss: 0.390275
[29719]	valid_0's binary_logloss: 0.390275
[29720]	valid_0's binary_logloss: 0.390275
[29721]	valid_0's binary_logloss: 0.390275
[29722]	valid_0's binary_logloss: 0.390275
[29723]	valid_0's binary_logloss: 0.390275
[29724]	valid_0's binary_logloss: 0.390275
[29725]	valid_0's binary_logloss: 0.390275
[29726]	valid_0's binary_logloss: 0.390275
[29727]	valid_0's binary_logloss: 0.390275
[29728]	valid_0's binary_logloss: 0.390275
[29729]	valid_0's binary_logloss: 0.390275
[29730]	val

[29898]	valid_0's binary_logloss: 0.39026
[29899]	valid_0's binary_logloss: 0.39026
[29900]	valid_0's binary_logloss: 0.39026
[29901]	valid_0's binary_logloss: 0.39026
[29902]	valid_0's binary_logloss: 0.39026
[29903]	valid_0's binary_logloss: 0.39026
[29904]	valid_0's binary_logloss: 0.39026
[29905]	valid_0's binary_logloss: 0.39026
[29906]	valid_0's binary_logloss: 0.39026
[29907]	valid_0's binary_logloss: 0.39026
[29908]	valid_0's binary_logloss: 0.39026
[29909]	valid_0's binary_logloss: 0.390259
[29910]	valid_0's binary_logloss: 0.390259
[29911]	valid_0's binary_logloss: 0.390259
[29912]	valid_0's binary_logloss: 0.390259
[29913]	valid_0's binary_logloss: 0.390259
[29914]	valid_0's binary_logloss: 0.390259
[29915]	valid_0's binary_logloss: 0.390259
[29916]	valid_0's binary_logloss: 0.390259
[29917]	valid_0's binary_logloss: 0.390259
[29918]	valid_0's binary_logloss: 0.390259
[29919]	valid_0's binary_logloss: 0.390259
[29920]	valid_0's binary_logloss: 0.390259
[29921]	valid_0's bina

In [21]:
gbm.save_model('lightgbm_20170826.txt')

In [1]:
import lightgbm as lgb
import numpy as np
#restore model
gbm = lgb.Booster(model_file='lightgbm_20170826.txt')


In [2]:
X_test_features = np.load('test_xgb_hash.npy')

In [5]:
y_hat = gbm.predict(X_test_features,num_iteration=10000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170825_10000.csv",index=False)

In [6]:
y_hat = gbm.predict(X_test_features,num_iteration=12500) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170825_12500.csv",index=False)

In [7]:
y_hat = gbm.predict(X_test_features,num_iteration=23000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170825_23000.csv",index=False)

In [15]:
y_hat = gbm.predict(X_test_features,num_iteration=3000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170825_3000.csv",index=False)

## [Sec3-2] XGBoost

In [2]:
import xgboost as xgb
param = {'max_depth':6, 'eta':0.01, 'silent':1, 
         'objective':'binary:logistic' ,'subsample':0.3,
        'min_child_weight':2,'lambda':2,'tree_method':'approx'}
param['nthread'] = 4
param['colsample_bytree'] = 0.5
param['eval_metric'] = 'logloss'

#msk = np.random.rand(len(X_train)) < 0.8
msk = np.load("20170727_msk.npy")

dtrain = xgb.DMatrix(X_train_features[msk], labels[msk])
dvali = xgb.DMatrix(X_train_features[~msk], labels[~msk])

/home/stream/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
num_round = 30000
evallist  = [ (dtrain,'train'),(dvali,'eval')]
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=1000)

[13175]	train-logloss:0.392896	eval-logloss:0.393222
[13176]	train-logloss:0.392896	eval-logloss:0.393222
[13177]	train-logloss:0.392896	eval-logloss:0.393222
[13178]	train-logloss:0.392896	eval-logloss:0.393221
[13179]	train-logloss:0.392895	eval-logloss:0.393221
[13180]	train-logloss:0.392895	eval-logloss:0.393221
[13181]	train-logloss:0.392895	eval-logloss:0.393221
[13182]	train-logloss:0.392895	eval-logloss:0.393221
[13183]	train-logloss:0.392895	eval-logloss:0.393221
[13184]	train-logloss:0.392894	eval-logloss:0.393221
[13185]	train-logloss:0.392894	eval-logloss:0.39322
[13186]	train-logloss:0.392894	eval-logloss:0.39322
[13187]	train-logloss:0.392894	eval-logloss:0.39322
[13188]	train-logloss:0.392893	eval-logloss:0.39322
[13189]	train-logloss:0.392893	eval-logloss:0.39322
[13190]	train-logloss:0.392893	eval-logloss:0.393219
[13191]	train-logloss:0.392893	eval-logloss:0.393219
[13192]	train-logloss:0.392893	eval-logloss:0.393219
[13193]	train-logloss:0.392892	eval-logloss:0.39321

[13331]	train-logloss:0.392864	eval-logloss:0.393195
[13332]	train-logloss:0.392864	eval-logloss:0.393195
[13333]	train-logloss:0.392863	eval-logloss:0.393195
[13334]	train-logloss:0.392863	eval-logloss:0.393195
[13335]	train-logloss:0.392863	eval-logloss:0.393195
[13336]	train-logloss:0.392863	eval-logloss:0.393195
[13337]	train-logloss:0.392863	eval-logloss:0.393195
[13338]	train-logloss:0.392863	eval-logloss:0.393194
[13339]	train-logloss:0.392862	eval-logloss:0.393194
[13340]	train-logloss:0.392862	eval-logloss:0.393194
[13341]	train-logloss:0.392862	eval-logloss:0.393194
[13342]	train-logloss:0.392862	eval-logloss:0.393194
[13343]	train-logloss:0.392861	eval-logloss:0.393193
[13344]	train-logloss:0.392861	eval-logloss:0.393193
[13345]	train-logloss:0.392861	eval-logloss:0.393193
[13346]	train-logloss:0.392861	eval-logloss:0.393193
[13347]	train-logloss:0.392861	eval-logloss:0.393193
[13348]	train-logloss:0.392861	eval-logloss:0.393193
[13349]	train-logloss:0.392861	eval-logloss:0.

[13487]	train-logloss:0.392832	eval-logloss:0.393169
[13488]	train-logloss:0.392832	eval-logloss:0.393169
[13489]	train-logloss:0.392832	eval-logloss:0.393169
[13490]	train-logloss:0.392831	eval-logloss:0.393169
[13491]	train-logloss:0.392831	eval-logloss:0.393169
[13492]	train-logloss:0.392831	eval-logloss:0.393169
[13493]	train-logloss:0.392831	eval-logloss:0.393169
[13494]	train-logloss:0.392831	eval-logloss:0.393169
[13495]	train-logloss:0.392831	eval-logloss:0.393168
[13496]	train-logloss:0.39283	eval-logloss:0.393168
[13497]	train-logloss:0.39283	eval-logloss:0.393168
[13498]	train-logloss:0.39283	eval-logloss:0.393168
[13499]	train-logloss:0.39283	eval-logloss:0.393168
[13500]	train-logloss:0.39283	eval-logloss:0.393167
[13501]	train-logloss:0.39283	eval-logloss:0.393167
[13502]	train-logloss:0.392829	eval-logloss:0.393167
[13503]	train-logloss:0.392829	eval-logloss:0.393167
[13504]	train-logloss:0.392829	eval-logloss:0.393166
[13505]	train-logloss:0.392828	eval-logloss:0.393166

[13643]	train-logloss:0.392797	eval-logloss:0.39314
[13644]	train-logloss:0.392797	eval-logloss:0.39314
[13645]	train-logloss:0.392797	eval-logloss:0.39314
[13646]	train-logloss:0.392797	eval-logloss:0.39314
[13647]	train-logloss:0.392796	eval-logloss:0.39314
[13648]	train-logloss:0.392796	eval-logloss:0.39314
[13649]	train-logloss:0.392796	eval-logloss:0.393139
[13650]	train-logloss:0.392796	eval-logloss:0.393139
[13651]	train-logloss:0.392796	eval-logloss:0.393139
[13652]	train-logloss:0.392796	eval-logloss:0.393139
[13653]	train-logloss:0.392795	eval-logloss:0.393139
[13654]	train-logloss:0.392795	eval-logloss:0.393139
[13655]	train-logloss:0.392794	eval-logloss:0.393138
[13656]	train-logloss:0.392794	eval-logloss:0.393138
[13657]	train-logloss:0.392794	eval-logloss:0.393138
[13658]	train-logloss:0.392794	eval-logloss:0.393138
[13659]	train-logloss:0.392794	eval-logloss:0.393137
[13660]	train-logloss:0.392793	eval-logloss:0.393137
[13661]	train-logloss:0.392793	eval-logloss:0.393137

[13799]	train-logloss:0.392763	eval-logloss:0.393112
[13800]	train-logloss:0.392763	eval-logloss:0.393112
[13801]	train-logloss:0.392763	eval-logloss:0.393112
[13802]	train-logloss:0.392763	eval-logloss:0.393112
[13803]	train-logloss:0.392763	eval-logloss:0.393112
[13804]	train-logloss:0.392763	eval-logloss:0.393111
[13805]	train-logloss:0.392762	eval-logloss:0.393111
[13806]	train-logloss:0.392762	eval-logloss:0.393111
[13807]	train-logloss:0.392762	eval-logloss:0.393111
[13808]	train-logloss:0.392762	eval-logloss:0.393111
[13809]	train-logloss:0.392762	eval-logloss:0.393111
[13810]	train-logloss:0.392762	eval-logloss:0.393111
[13811]	train-logloss:0.392761	eval-logloss:0.393111
[13812]	train-logloss:0.392761	eval-logloss:0.39311
[13813]	train-logloss:0.392761	eval-logloss:0.39311
[13814]	train-logloss:0.392761	eval-logloss:0.39311
[13815]	train-logloss:0.392761	eval-logloss:0.39311
[13816]	train-logloss:0.39276	eval-logloss:0.39311
[13817]	train-logloss:0.39276	eval-logloss:0.39311
[

...

[14735]	train-logloss:0.392569	eval-logloss:0.392952
[14736]	train-logloss:0.392569	eval-logloss:0.392952
[14737]	train-logloss:0.392568	eval-logloss:0.392951
[14738]	train-logloss:0.392568	eval-logloss:0.392951
[14739]	train-logloss:0.392568	eval-logloss:0.392951
[14740]	train-logloss:0.392568	eval-logloss:0.392951
[14741]	train-logloss:0.392567	eval-logloss:0.392951
[14742]	train-logloss:0.392567	eval-logloss:0.392951
[14743]	train-logloss:0.392567	eval-logloss:0.392951
[14744]	train-logloss:0.392567	eval-logloss:0.39295
[14745]	train-logloss:0.392567	eval-logloss:0.39295
[14746]	train-logloss:0.392567	eval-logloss:0.39295
[14747]	train-logloss:0.392567	eval-logloss:0.39295
[14748]	train-logloss:0.392566	eval-logloss:0.39295
[14749]	train-logloss:0.392566	eval-logloss:0.39295
[14750]	train-logloss:0.392566	eval-logloss:0.392949
[14751]	train-logloss:0.392565	eval-logloss:0.392949
[14752]	train-logloss:0.392565	eval-logloss:0.392949
[14753]	train-logloss:0.392565	eval-logloss:0.392949

[14891]	train-logloss:0.392537	eval-logloss:0.392926
[14892]	train-logloss:0.392537	eval-logloss:0.392926
[14893]	train-logloss:0.392537	eval-logloss:0.392926
[14894]	train-logloss:0.392536	eval-logloss:0.392926
[14895]	train-logloss:0.392536	eval-logloss:0.392926
[14896]	train-logloss:0.392536	eval-logloss:0.392925
[14897]	train-logloss:0.392536	eval-logloss:0.392925
[14898]	train-logloss:0.392536	eval-logloss:0.392925
[14899]	train-logloss:0.392535	eval-logloss:0.392925
[14900]	train-logloss:0.392535	eval-logloss:0.392925
[14901]	train-logloss:0.392535	eval-logloss:0.392925
[14902]	train-logloss:0.392535	eval-logloss:0.392924
[14903]	train-logloss:0.392535	eval-logloss:0.392924
[14904]	train-logloss:0.392534	eval-logloss:0.392924
[14905]	train-logloss:0.392534	eval-logloss:0.392924
[14906]	train-logloss:0.392534	eval-logloss:0.392924
[14907]	train-logloss:0.392534	eval-logloss:0.392924
[14908]	train-logloss:0.392534	eval-logloss:0.392924
[14909]	train-logloss:0.392534	eval-logloss:0.

[15047]	train-logloss:0.392507	eval-logloss:0.392902
[15048]	train-logloss:0.392507	eval-logloss:0.392902
[15049]	train-logloss:0.392507	eval-logloss:0.392902
[15050]	train-logloss:0.392507	eval-logloss:0.392902
[15051]	train-logloss:0.392506	eval-logloss:0.392901
[15052]	train-logloss:0.392506	eval-logloss:0.392901
[15053]	train-logloss:0.392506	eval-logloss:0.392901
[15054]	train-logloss:0.392506	eval-logloss:0.392901
[15055]	train-logloss:0.392505	eval-logloss:0.392901
[15056]	train-logloss:0.392505	eval-logloss:0.392901
[15057]	train-logloss:0.392505	eval-logloss:0.3929
[15058]	train-logloss:0.392505	eval-logloss:0.3929
[15059]	train-logloss:0.392505	eval-logloss:0.3929
[15060]	train-logloss:0.392505	eval-logloss:0.3929
[15061]	train-logloss:0.392505	eval-logloss:0.3929
[15062]	train-logloss:0.392505	eval-logloss:0.3929
[15063]	train-logloss:0.392504	eval-logloss:0.3929
[15064]	train-logloss:0.392504	eval-logloss:0.3929
[15065]	train-logloss:0.392504	eval-logloss:0.392899
[15066]	t

[15203]	train-logloss:0.39248	eval-logloss:0.39288
[15204]	train-logloss:0.39248	eval-logloss:0.39288
[15205]	train-logloss:0.392479	eval-logloss:0.39288
[15206]	train-logloss:0.392479	eval-logloss:0.39288
[15207]	train-logloss:0.392479	eval-logloss:0.39288
[15208]	train-logloss:0.392479	eval-logloss:0.39288
[15209]	train-logloss:0.392479	eval-logloss:0.39288
[15210]	train-logloss:0.392478	eval-logloss:0.392879
[15211]	train-logloss:0.392478	eval-logloss:0.392879
[15212]	train-logloss:0.392478	eval-logloss:0.392879
[15213]	train-logloss:0.392478	eval-logloss:0.392879
[15214]	train-logloss:0.392478	eval-logloss:0.392879
[15215]	train-logloss:0.392478	eval-logloss:0.392879
[15216]	train-logloss:0.392478	eval-logloss:0.392879
[15217]	train-logloss:0.392477	eval-logloss:0.392879
[15218]	train-logloss:0.392477	eval-logloss:0.392878
[15219]	train-logloss:0.392477	eval-logloss:0.392878
[15220]	train-logloss:0.392477	eval-logloss:0.392878
[15221]	train-logloss:0.392477	eval-logloss:0.392878
[1

[15359]	train-logloss:0.392452	eval-logloss:0.392858
[15360]	train-logloss:0.392452	eval-logloss:0.392858
[15361]	train-logloss:0.392451	eval-logloss:0.392858
[15362]	train-logloss:0.392451	eval-logloss:0.392857
[15363]	train-logloss:0.392451	eval-logloss:0.392857
[15364]	train-logloss:0.392451	eval-logloss:0.392857
[15365]	train-logloss:0.39245	eval-logloss:0.392857
[15366]	train-logloss:0.39245	eval-logloss:0.392856
[15367]	train-logloss:0.39245	eval-logloss:0.392856
[15368]	train-logloss:0.39245	eval-logloss:0.392856
[15369]	train-logloss:0.392449	eval-logloss:0.392856
[15370]	train-logloss:0.392449	eval-logloss:0.392856
[15371]	train-logloss:0.392449	eval-logloss:0.392856
[15372]	train-logloss:0.392449	eval-logloss:0.392856
[15373]	train-logloss:0.392449	eval-logloss:0.392856
[15374]	train-logloss:0.392448	eval-logloss:0.392855
[15375]	train-logloss:0.392448	eval-logloss:0.392855
[15376]	train-logloss:0.392448	eval-logloss:0.392855
[15377]	train-logloss:0.392448	eval-logloss:0.3928

In [ ]:
bst.best_score

In [ ]:
bst.save_model('md_hash_0815.model')

## TEsting 

In [ ]:
import xgboost as xgb

In [ ]:
bst = xgb.Booster({'nthread':4}) #init model
bst.load_model("md_hash_0815.model") # load model
dtest = xgb.DMatrix(tmp_features)
ypred = bst.predict(dtest,ntree_limit=300)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred 
sampleSubmission.to_csv("md_hash_0815_300.csv",index=False)


In [ ]:
bst.get_fscore()

In [ ]:
%matplotlib inline

xgb.plot_importance(bst)